In [ ]:
from pathlib import Path
import numpy as np
from aicsimageio import AICSImage
from aicsimageio.readers.ome_tiff_reader import OmeTiffReader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

pixel_size = 0.227

In [ ]:
input_dir = Path(input())

In [ ]:
exp_dirs = [exp_dir for exp_dir in input_dir.glob('*aligned')]
exp_dirs.sort()

In [ ]:
def get_ch_measurements(imgpath, d):
    img_file = AICSImage(imgpath, reader=OmeTiffReader)
    img = img_file.data
    masked_img = np.ma.masked_array(img, img==0)
        
    min_chs = np.ma.min(masked_img, axis=(0, 2, 3, 4))
    median_chs = np.ma.median(masked_img, axis=(0, 2, 3, 4))
    mean_chs = np.ma.mean(masked_img, axis=(0, 2, 3, 4))
    max_chs = np.ma.max(masked_img, axis=(0, 2, 3, 4))
    intdens_chs = np.ma.sum(masked_img, axis=(0, 2, 3, 4))
    
    
    if len(d)==0:
        d['Image name'] = [imgpath.name]
        d['Cell area (pixels)'] = [np.count_nonzero(img[0, 0, 0, :, :])]
        
        for ch in range(img.shape[1]):
            d[f'min_ch{ch}'] = [min_chs[ch]]
            d[f'median_ch{ch}'] = [median_chs[ch]]
            d[f'mean_ch{ch}'] = [mean_chs[ch]]
            d[f'max_ch{ch}'] = [max_chs[ch]]
            d[f'intdens_ch{ch}'] = [intdens_chs[ch]]
        
    else:
        d['Image name'].append(imgpath.name)
        d['Cell area (pixels)'].append(np.count_nonzero(img[0, 0, 0, :, :]))
    
        for ch in range(img.shape[1]):
            d[f'min_ch{ch}'].append(min_chs[ch])
            d[f'median_ch{ch}'].append(median_chs[ch])
            d[f'mean_ch{ch}'].append(mean_chs[ch])
            d[f'max_ch{ch}'].append(max_chs[ch])
            d[f'intdens_ch{ch}'].append(intdens_chs[ch])
    return d


d = {}
for exp_dir in exp_dirs:
    print(f'Analyzing {exp_dir.name}')
    img_dir = exp_dir / 'img_processing' / 'masked_imgs' / 'raw_ometifs_subset_masked_cellmasks'
    assert img_dir.is_dir()
    # Get all masked raw-tiff images in the image directory
    imgpaths = [imgpath for imgpath in img_dir.glob('*.ome.tif')]
    imgpaths.sort()

    for i, imgpath in enumerate(imgpaths):
        d = get_ch_measurements(imgpath, d)
    df = pd.DataFrame(d)
    df.to_csv(input_dir / f'ch_int_measurements.csv')


In [ ]:
def get_ch_measurements(imgpath, d):
    img_file = AICSImage(imgpath, reader=OmeTiffReader)
    img = img_file.data
    masked_img = np.ma.masked_array(img, img==0)
        
    min_chs = np.ma.min(masked_img, axis=(0, 2, 3, 4))
    median_chs = np.ma.median(masked_img, axis=(0, 2, 3, 4))
    mean_chs = np.ma.mean(masked_img, axis=(0, 2, 3, 4))
    max_chs = np.ma.max(masked_img, axis=(0, 2, 3, 4))
    intdens_chs = np.ma.sum(masked_img, axis=(0, 2, 3, 4))
    
    
    if len(d)==0:
        d['Image name'] = [imgpath.name]
        
        for ch in range(img.shape[1]):
            d[f'min_ch{ch}_nomask'] = [min_chs[ch]]
            d[f'median_ch{ch}_nomask'] = [median_chs[ch]]
            d[f'mean_ch{ch}_nomask'] = [mean_chs[ch]]
            d[f'max_ch{ch}_nomask'] = [max_chs[ch]]
            d[f'intdens_ch{ch}_nomask'] = [intdens_chs[ch]]
        
    else:
        d['Image name'].append(imgpath.name)
    
        for ch in range(img.shape[1]):
            d[f'min_ch{ch}_nomask'].append(min_chs[ch])
            d[f'median_ch{ch}_nomask'].append(median_chs[ch])
            d[f'mean_ch{ch}_nomask'].append(mean_chs[ch])
            d[f'max_ch{ch}_nomask'].append(max_chs[ch])
            d[f'intdens_ch{ch}_nomask'].append(intdens_chs[ch])
    return d


d = {}
for exp_dir in exp_dirs:
    print(f'Analyzing {exp_dir.name}')
    img_dir = exp_dir / 'img_processing' / 'raw_ometifs_subset'
    assert img_dir.is_dir()
    # Get all masked raw-tiff images in the image directory
    imgpaths = [imgpath for imgpath in img_dir.glob('*.ome.tif')]
    imgpaths.sort()

    for i, imgpath in enumerate(imgpaths):
        d = get_ch_measurements(imgpath, d)
    df = pd.DataFrame(d)
    df.to_csv(input_dir / f'ch_int_measurements_nomask.csv')

In [ ]:
df = pd.read_csv(input_dir / f'ch_int_measurements.csv')
df_nomask = pd.read_csv(input_dir / f'ch_int_measurements_nomask.csv')
df['Image basename'] = df['Image name'].str.split('_2023').str[0]
df['Image basename']
df_nomask['Image basename'] = df_nomask['Image name'].str.split('.ome.tif').str[0]
df_nomask['Image basename']
merge_df = df.merge(df_nomask, how='outer', on='Image basename')
merge_df.to_csv(input_dir / 'merge.csv')


In [ ]:
merge_df = merge_df.dropna(axis=0)
merge_df['Mean phalloidin intensity / cell (au)'] = (merge_df['mean_ch3'] - merge_df['median_ch3_nomask'])
merge_df['Median phalloidin intensity / cell (au)'] = (merge_df['mean_ch3'] - merge_df['median_ch3_nomask'])

In [ ]:
df

In [ ]:
df = pd.read_csv(input_dir / 'combined' / 'merge.csv')
wells = [df['Well']=='I5-A1', df['Well']=='I5-A2', df['Well']=='I5-A3', df['Well']=='I5-B1', df['Well']=='I5-B3', df['Well']=='I6-A2']
conditions = ['StablAct', 'Control', 'Control', 'DeAct', 'DeAct', 'StablAct']
replicate = [1, 1, 2, 1, 2, 2]
df['Condition'] = np.select(wells, conditions)
df['Condition'] = np.select(wells, conditions)
df['% exclusion / cell'] = df['% exclusion / cell']*100
df.to_csv(input_dir / 'combined' / 'merge.csv', index=False)

In [ ]:
plt.rcParams.update({'font.size': 12})
x_label = 'Mean phalloidin intensity / cell (au)'
y_label = '% exclusion / cell'
df_to_plot = df.where(df['Exclude']!='Y')
sns.scatterplot(df_to_plot.where(df['Condition']=='Control'), x=x_label, y=y_label, color='gray')
plt.suptitle('Control')
plt.savefig(input_dir / 'combined' / 'exclusionvsphal_ctrl.png', bbox_inches='tight')
plt.show()
sns.scatterplot(df_to_plot.where(df['Condition']=='DeAct'), x=x_label, y=y_label, color='gray')
plt.suptitle('DeAct')
plt.savefig(input_dir / 'combined' / 'exclusionvsphal_deact.png', bbox_inches='tight')
plt.show()
sns.scatterplot(df_to_plot.where(df['Condition']=='StablAct'), x=x_label, y=y_label, color='gray')
plt.suptitle('StablAct')
plt.savefig(input_dir / 'combined' / 'exclusionvsphal_stablact.png', bbox_inches='tight')
plt.show()
sns.scatterplot(df_to_plot, x=x_label, y=y_label, color='gray')
plt.suptitle('all')
plt.savefig(input_dir / 'combined' / 'exclusionvsphal.png', bbox_inches='tight')
plt.show()


In [ ]:
sns.scatterplot(df_to_plot.where(df['Condition']=='StablActs'), x=x_label, y=y_label, color='gray')
plt.savefig(input_dir / 'combined' / 'exclusionvsphal_ctrl.png')
plt.show()

In [ ]:
y_label = 'Mean phalloidin intensity / cell (au)'
y_label_save = 'phal_int.png'
df_to_plot = df.where(df['Exclude']!='Y')
ymax=40

plt.rcParams.update({'font.size': 13})
plt.figure(figsize=(5,6))
x_order = ['Control', 'DeAct', 'StablAct']
x_label = 'Condition'

ax = sns.pointplot(df_to_plot, x=x_label, y=y_label, order=x_order, hue='Replicate', color='k', dodge=True, errorbar=None, linestyles='None')
ax.set(ylim=(0, ymax))
ax.collections[0].set_sizes([20]) 
ax.collections[1].set_sizes([20]) 

ax = sns.swarmplot(df_to_plot, x=x_label, y=y_label, order=x_order, legend=False, color='gray', alpha=0.4)
ax.collections[2].set_sizes([10]) 
ax.collections[3].set_sizes([10]) 
ax.collections[4].set_sizes([10]) 
ax.get_legend().remove()
plt.savefig(input_dir / 'combined' / y_label_save, bbox_inches='tight')

